In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
VWAP + VWMA(21) Intraday Backtester (based on your ORB framework)
=================================================================

Strategy Overview
-----------------
Implements **Strategy #3: VWAP + VWMA** exactly as described in the transcript.

**Indicators**
- VWAP (session VWAP; resets every day at 09:15 IST)
- VWMA(21) (21 bars on 5-minute timeframe; 20 acceptable but we use 21 here)

**Why combine them?**
Both are trusted, volume-aware *lagging* indicators. When **VWAP and VWMA are
very close or meeting**, that zone becomes a high-probability reaction area.

**Candle Roles (reused from VWAP-only lecture)**
1) **Opening Candle**: first candle that **closes above BOTH lines** (bullish) or
   **closes below BOTH lines** (bearish).
2) **Signal Candle**: the *next* candle that **breaks the high/low of the Opening candle**.
3) **Entry Candle**: the *next* candle that **breaks the high/low of the Signal candle**.
   → We execute at the **next bar open** (more realistic fills).

**Mandatory Proximity Rules (from transcript)**
- **Lines proximity**: VWAP and VWMA must be **very close or meeting** at/near the setup.
- **Entry proximity**: the Entry candle must be **very close to BOTH lines**.
- If the Entry candle is far from the two lines, **skip**.
- If VWAP and VWMA are far apart, **skip**.

**Trade Direction**
- Close above BOTH lines ⇒ bullish sequence (Opening → Signal↑ → Entry↑).
- Close below BOTH lines ⇒ bearish sequence (Opening → Signal↓ → Entry↓).

**Risk & Money Management**
- One trade **per day per stock**.
- Intraday leverage supported (default 5×).
- Absolute rupee SL/TP **per position** (default: SL ₹3,000; TP ₹11,000 in this template; tune as you like).
- Optional **Trailing SL** (rupee distance per position).
- Force **square-off** near close (default 14:45 IST here).

**Costs**
Full **Groww Intraday (NSE)** cost model retained:
Brokerage, STT, Stamp duty, Exchange, SEBI, IPFT, GST — applied per round trip.

**Outputs**
- `trades.csv` with complete details (entry/exit, qty, gross, charges, net, reason).
- Console summary (overall, by ticker, by direction) + max drawdown on net P&L.

---------------------------------------------------------------------------
Tunable proximity thresholds (based on transcript's "very close"):
- `lines_proximity_pct`: max allowed |VWAP - VWMA| as % of price (default 0.15%).
- `entry_proximity_pct`: max allowed distance of Entry candle (its OPEN) from BOTH lines
  as % of price (default 0.20%).
Adjust per your instruments/volatility if you want more/less selectivity.
---------------------------------------------------------------------------
"""

import math
import sys
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import pandas as pd
import numpy as np

try:
    import yfinance as yf
except Exception:
    print("Please: pip install yfinance pandas numpy")
    sys.exit(1)

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    tickers: List[str]
    start_date: str = "2025-10-01"
    end_date:   str = "2025-11-01"
    interval:   str = "5m"

    capital_per_stock: float = 100_000.0   # your cash per symbol
    intraday_leverage: float = 5.0         # 5× buying power (intraday)

    # Absolute ₹ stop/target per POSITION (will be divided by qty to get per-share)
    sl_rupees: float = 2000.0
    tp_rupees: float = 6000.0

    # Direction toggles
    enable_longs: bool = True
    enable_shorts: bool = False

    # Trailing SL
    enable_trailing_sl: bool = False
    trail_rupees: float = 3000.0

    # Trading session (IST)
    entry_start: str = "10:00"              # allow setups after 10:00
    entry_end:   str = "14:00"              # last time to open a trade
    squareoff_time: str = "14:00"  

    # Proximity thresholds (as fractions of price, e.g., 0.0015 = 0.15%)
    lines_proximity_pct: float = 0.0015
    entry_proximity_pct: float = 0.0020

    # Require that all three candles (Opening, Signal, Entry) occur while
    # VWAP/VWMA are within proximity? (stricter filter)
    require_proximity_on_all: bool = True

    timezone: str = "Asia/Kolkata"
    out_file: str = "trades.csv"

CFG = Config(
    tickers=[
        'ADANIENT.NS','ADANIPORTS.NS','APOLLOHOSP.NS','ASIANPAINT.NS','AXISBANK.NS',
        'BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BEL.NS','BHARTIARTL.NS',
        'CIPLA.NS','COALINDIA.NS','DRREDDY.NS','EICHERMOT.NS','GRASIM.NS','HCLTECH.NS',
        'HDFCBANK.NS','HDFCLIFE.NS','HINDALCO.NS','HINDUNILVR.NS','ICICIBANK.NS',
        'ITC.NS','INFY.NS','INDIGO.NS','JSWSTEEL.NS','JIOFIN.NS','KOTAKBANK.NS',
        'LT.NS','M&M.NS','MARUTI.NS','NTPC.NS','NESTLEIND.NS','ONGC.NS',
        'POWERGRID.NS','RELIANCE.NS','SBILIFE.NS','SHRIRAMFIN.NS','SBIN.NS',
        'SUNPHARMA.NS','TCS.NS','TATACONSUM.NS','TMPV.NS','TATASTEEL.NS',
        'TECHM.NS','TITAN.NS','TRENT.NS','ULTRACEMCO.NS','WIPRO.NS'
    ]
)

# =========================
# GROWW CHARGES (NSE intraday)
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    """
    Compute charges for a single round-trip intraday equity trade (NSE).
    """
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)  # floor ₹5

    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)

    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover

    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover

    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover

    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)

    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover

    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)

    return {
        "brokerage_buy": bro_buy, "brokerage_sell": bro_sell,
        "exchange_buy": exch_buy, "exchange_sell": exch_sell,
        "sebi_buy": sebi_buy, "sebi_sell": sebi_sell,
        "ipft_buy": ipft_buy, "ipft_sell": ipft_sell,
        "gst_buy": gst_buy, "gst_sell": gst_sell,
        "stt_sell": stt_sell, "stamp_buy": stamp_buy,
        "total_charges": total
    }

# =========================
# DATA
# =========================
def fetch_5m_dataframe(ticker: str, start: str, end: str, interval: str, tz: str) -> pd.DataFrame:
    df = yf.download(
        ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=False,
        prepost=False,
        progress=False,
        multi_level_index=False,
        group_by="column",
    )
    if df.empty:
        return df

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([c for c in col if c]).strip() for col in df.columns.values]

    # IST timezone
    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert(tz)
    else:
        df = df.tz_convert(tz)

    cols_map = {c: c.title() for c in df.columns}
    df.rename(columns=cols_map, inplace=True)
    keep = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
    df = df[keep]
    df["Ticker"] = ticker
    return df

def session_filter(df: pd.DataFrame, date: pd.Timestamp, tz: str) -> pd.DataFrame:
    day = pd.Timestamp(date)
    day_start = pd.Timestamp(day.date().strftime("%Y-%m-%d") + " 09:15").tz_localize(tz)
    day_end   = pd.Timestamp(day.date().strftime("%Y-%m-%d") + " 15:30").tz_localize(tz)
    return df.loc[(df.index >= day_start) & (df.index <= day_end)].copy()

# =========================
# INDICATORS: Session VWAP + VWMA(21)
# =========================
def add_session_vwap_and_vwma(day_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds:
      - 'VWAP': session VWAP using Typical Price (H+L+C)/3
      - 'VWMA21': rolling 21-bar Volume Weighted MA on Close
    Assumes day_df is a single-session slice (09:15..15:30 IST).
    """
    df = day_df.copy()
    tp = (df["High"] + df["Low"] + df["Close"]) / 3.0
    cum_pv = (tp * df["Volume"]).cumsum()
    cum_v  = df["Volume"].cumsum()
    df["VWAP"] = np.where(cum_v > 0, cum_pv / cum_v, np.nan)

    # VWMA(21) on Close
    w_close = (df["Close"] * df["Volume"]).rolling(21, min_periods=1).sum()
    w_vol   = df["Volume"].rolling(21, min_periods=1).sum()
    df["VWMA21"] = np.where(w_vol > 0, w_close / w_vol, np.nan)

    return df

# =========================
# HELPERS
# =========================
def _ts_on_day(day: pd.Timestamp, hhmm: str, tz: str) -> pd.Timestamp:
    return pd.Timestamp(day.date().strftime("%Y-%m-%d") + f" {hhmm}").tz_localize(tz)

def _within_pct(value_diff: float, ref_price: float, max_pct: float) -> bool:
    if not np.isfinite(value_diff) or not np.isfinite(ref_price) or ref_price <= 0:
        return False
    return abs(value_diff) <= (max_pct * ref_price)

def _lines_close_enough(row: pd.Series, lines_proximity_pct: float) -> bool:
    price = float(row["Close"])
    return _within_pct(float(row["VWAP"] - row["VWMA21"]), price, lines_proximity_pct)

def _entry_candle_close_to_both(entry_open: float, vwap: float, vwma: float,
                                price_ref: float, entry_proximity_pct: float) -> bool:
    # Must be close to BOTH lines
    return (_within_pct(entry_open - vwap, price_ref, entry_proximity_pct) and
            _within_pct(entry_open - vwma, price_ref, entry_proximity_pct))

# =========================
# VWAP+VWMA LOGIC (Opening → Signal → Entry)
# =========================
def find_vwap_vwma_sequence(day_df: pd.DataFrame,
                            day: pd.Timestamp,
                            cfg: Config) -> Tuple[Optional[str], Optional[pd.Timestamp], Optional[float]]:
    """
    Returns (direction, entry_ts, entry_price) or (None, None, None)

    Steps:
      1) Opening candle: first candle after entry_start whose CLOSE is above BOTH (long) or below BOTH (short)
         AND lines proximity holds at that moment.
      2) Signal candle: next candle that breaks the high (long) or low (short) of the Opening candle.
      3) Entry candle: next candle that breaks the high/low of the Signal candle.
         → Execute at the NEXT bar OPEN (realistic).
      4) Entry must be very close to BOTH lines (entry proximity).
      5) Respect direction toggles and entry_end cutoff.
    """
    start_ts = _ts_on_day(day, cfg.entry_start, cfg.timezone)
    end_last_entry = _ts_on_day(day, cfg.entry_end, cfg.timezone)
    df = day_df.loc[day_df.index >= start_ts].copy()

    if len(df) < 25:
        return None, None, None

    # 1) Find Opening candle
    opening_idx = None
    opening_dir = None

    for i in range(len(df)):
        row = df.iloc[i]
        close = float(row["Close"])
        vwap = float(row["VWAP"])
        vwma = float(row["VWMA21"])

        # Lines must be close
        if not _lines_close_enough(row, cfg.lines_proximity_pct):
            continue

        # Above BOTH or below BOTH
        if close > max(vwap, vwma) and cfg.enable_longs:
            opening_idx = i
            opening_dir = "long"
            break
        if close < min(vwap, vwma) and cfg.enable_shorts:
            opening_idx = i
            opening_dir = "short"
            break

    if opening_idx is None:
        return None, None, None

    opening_row = df.iloc[opening_idx]
    opening_high = float(opening_row["High"])
    opening_low  = float(opening_row["Low"])

    # 2) Signal candle: next candle breaking Opening high/low
    signal_idx = None
    for i in range(opening_idx + 1, len(df)):
        row = df.iloc[i]
        if cfg.require_proximity_on_all and not _lines_close_enough(row, cfg.lines_proximity_pct):
            continue
        if opening_dir == "long" and float(row["High"]) > opening_high:
            signal_idx = i
            break
        if opening_dir == "short" and float(row["Low"]) < opening_low:
            signal_idx = i
            break

    if signal_idx is None:
        return None, None, None

    signal_row = df.iloc[signal_idx]
    signal_high = float(signal_row["High"])
    signal_low  = float(signal_row["Low"])

    # 3) Entry candle: next candle breaking Signal high/low
    entry_idx = None
    trigger = signal_high if opening_dir == "long" else signal_low

    for i in range(signal_idx + 1, len(df)):
        row = df.iloc[i]
        if cfg.require_proximity_on_all and not _lines_close_enough(row, cfg.lines_proximity_pct):
            continue

        broke = (float(row["High"]) > trigger) if opening_dir == "long" else (float(row["Low"]) < trigger)
        if not broke:
            continue

        # Candidate entry executes at NEXT bar OPEN
        if i + 1 >= len(df):
            return None, None, None
        next_row = df.iloc[i + 1]
        entry_ts = df.index[i + 1]
        if entry_ts > end_last_entry:
            return None, None, None

        entry_open = float(next_row["Open"])
        price_ref = float(row["Close"])  # or use entry_open as ref; close is fine for % scaling

        # Entry proximity: must be close to BOTH lines
        vwap_now = float(next_row["VWAP"]) if "VWAP" in next_row else float(row["VWAP"])
        vwma_now = float(next_row["VWMA21"]) if "VWMA21" in next_row else float(row["VWMA21"])
        if not _entry_candle_close_to_both(entry_open, vwap_now, vwma_now, price_ref, cfg.entry_proximity_pct):
            # Skip if far from lines (per transcript)
            continue

        # Direction toggles already respected at opening step, but double-check here
        if opening_dir == "long" and not cfg.enable_longs:
            return None, None, None
        if opening_dir == "short" and not cfg.enable_shorts:
            return None, None, None

        # For small gaps, keep trigger logic conservative:
        if opening_dir == "long":
            entry_price = max(entry_open, trigger)
        else:
            entry_price = min(entry_open, trigger)

        return opening_dir, entry_ts, float(entry_price)

    return None, None, None

# =========================
# SIMULATION (with trailing SL)
# =========================
def simulate_trade(day_df: pd.DataFrame, direction: str, entry_ts: pd.Timestamp, entry_price: float,
                   capital: float, leverage: float, sl_rupees: float, tp_rupees: float,
                   squareoff_time: str, tz: str, ticker: str,
                   enable_trailing_sl: bool, trail_rupees: float):
    if (entry_ts is None) or (not np.isfinite(entry_price)):
        return None

    buying_power = capital * leverage
    qty = int(buying_power // entry_price)
    if qty < 1:
        return None

    risk_per_share   = sl_rupees / qty
    target_per_share = tp_rupees / qty
    trail_per_share  = (trail_rupees if trail_rupees is not None else sl_rupees) / qty

    if direction == "long":
        static_sl = entry_price - risk_per_share
        tp = entry_price + target_per_share
        high_water = entry_price
        dyn_sl = static_sl
    else:
        static_sl = entry_price + risk_per_share
        tp = entry_price - target_per_share
        low_water = entry_price
        dyn_sl = static_sl

    exit_reason = None
    exit_ts = None
    exit_price = None

    after = day_df.loc[day_df.index >= entry_ts]
    for ts, row in after.iterrows():
        if enable_trailing_sl:
            if direction == "long":
                high_water = max(high_water, float(row["High"]))
                new_sl = max(static_sl, high_water - trail_per_share)
                dyn_sl = max(dyn_sl, new_sl)
            else:
                low_water = min(low_water, float(row["Low"]))
                new_sl = min(static_sl, low_water + trail_per_share)
                dyn_sl = min(dyn_sl, new_sl)

        if direction == "long":
            if row["Low"] <= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL",
                                                    ts, float(dyn_sl))
                break
            if row["High"] >= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break
        else:
            if row["High"] >= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL",
                                                    ts, float(dyn_sl))
                break
            if row["Low"] <= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break

        so = _ts_on_day(ts, squareoff_time, tz)
        if ts >= so:
            exit_reason, exit_ts, exit_price = "EOD", ts, float(row["Close"])
            break

    if exit_ts is None:
        last_ts = after.index[-1]
        exit_reason, exit_ts, exit_price = "EOD", last_ts, float(after.iloc[-1]["Close"])

    if direction == "long":
        gross = (exit_price - entry_price) * qty
        buy_turnover  = entry_price * qty
        sell_turnover = exit_price * qty
    else:
        gross = (entry_price - exit_price) * qty
        buy_turnover  = exit_price * qty
        sell_turnover = entry_price * qty

    fees = groww_intraday_charges(buy_turnover, sell_turnover)
    net = gross - fees["total_charges"]

    return {
        "date": entry_ts.date().isoformat(),
        "ticker": ticker,
        "direction": direction,
        "entry_time": entry_ts.isoformat(),
        "entry_price": round(entry_price, 2),
        "qty": qty,
        "sl_price_initial": round(static_sl, 2),
        "tp_price": round(tp, 2),
        "sl_trailing_enabled": enable_trailing_sl,
        "trail_rupees": trail_rupees,
        "exit_time": exit_ts.isoformat(),
        "exit_price": round(exit_price, 2),
        "gross_pnl": round(gross, 2),
        "charges": round(fees["total_charges"], 2),
        "net_pnl": round(net, 2),
        "exit_reason": exit_reason,
        "buy_turnover": round(buy_turnover, 2),
        "sell_turnover": round(sell_turnover, 2),
        "cash_capital": round(capital, 2),
        "leverage": leverage,
        "notional_at_entry": round(entry_price * qty, 2),
    }

# =========================
# BACKTEST
# =========================
def run_backtest(cfg: Config) -> pd.DataFrame:
    all_trades = []

    for ticker in cfg.tickers:
        print(f"Downloading {ticker} ...")
        df = fetch_5m_dataframe(ticker, cfg.start_date, cfg.end_date, cfg.interval, cfg.timezone)
        if df.empty:
            print(f"  WARN: No data for {ticker}")
            continue

        # Distinct session dates
        dates = sorted(list({pd.Timestamp(ts).date() for ts in df.index}))

        for d in dates:
            day = pd.Timestamp(d)
            day_df = session_filter(df, day, cfg.timezone)
            if len(day_df) < 30:
                continue

            # Add VWAP + VWMA(21) per session
            day_df = add_session_vwap_and_vwma(day_df)

            # Find (direction, entry_ts, entry_price) per transcript rules
            direction, en_ts, en_price = find_vwap_vwma_sequence(day_df, day, cfg)
            if direction is None or en_ts is None or not np.isfinite(en_price):
                continue

            # Simulate exactly ONE trade per day per ticker
            trade = simulate_trade(
                day_df=day_df, direction=direction,
                entry_ts=en_ts, entry_price=en_price,
                capital=cfg.capital_per_stock, leverage=cfg.intraday_leverage,
                sl_rupees=cfg.sl_rupees, tp_rupees=cfg.tp_rupees,
                squareoff_time=cfg.squareoff_time, tz=cfg.timezone,
                ticker=ticker,
                enable_trailing_sl=cfg.enable_trailing_sl,
                trail_rupees=cfg.trail_rupees
            )
            if trade:
                all_trades.append(trade)

    trades = pd.DataFrame(all_trades)
    if trades.empty:
        print("No trades generated.")
        return trades

    trades.sort_values(by=["date", "ticker", "entry_time"], inplace=True)
    trades.to_csv(cfg.out_file, index=False)
    return trades

# =========================
# METRICS
# =========================
def max_drawdown(series: pd.Series) -> float:
    cum = series.cumsum()
    peak = cum.cummax()
    dd = cum - peak
    return float(dd.min())

def summarize(trades: pd.DataFrame):
    print("\n=== OVERALL METRICS ===")
    n = len(trades)
    wins = (trades["net_pnl"] > 0).sum()
    win_rate = 100.0 * wins / n if n else 0.0
    gross = trades["gross_pnl"].sum()
    charges = trades["charges"].sum()
    net = trades["net_pnl"].sum()
    mdd = max_drawdown(trades["net_pnl"])

    print(f"Trades: {n} | Win rate: {win_rate:.1f}%")
    print(f"Gross P&L: ₹{gross:,.2f} | Charges: ₹{charges:,.2f} | Net P&L: ₹{net:,.2f}")
    print(f"Max Drawdown (net): ₹{mdd:,.2f}")

    print("\n=== BY TICKER ===")
    by_t = trades.groupby("ticker").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_t["win_rate"] = by_t["win_rate"].round(1)
    print(by_t.to_string(index=False))

    print("\n=== BY DIRECTION ===")
    by_dir = trades.groupby("direction").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_dir["win_rate"] = by_dir["win_rate"].round(1)
    print(by_dir.to_string(index=False))

    print("\nWrote trades to:", CFG.out_file)

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    trades = run_backtest(CFG)
    if not trades.empty:
        summarize(trades)



=== OVERALL METRICS ===
Trades: 497 | Win rate: 36.2%
Gross P&L: ₹-88,520.18 | Charges: ₹111,366.04 | Net P&L: ₹-199,886.15
Max Drawdown (net): ₹-235,269.11

=== BY TICKER ===
       ticker  n  wins     gross  charges       net  win_rate
  ADANIENT.NS  9     1 -12624.44  2013.55 -14637.99      11.1
ADANIPORTS.NS  8     3  -4352.39  1793.22  -6145.62      37.5
APOLLOHOSP.NS 13     5  -1463.50  2905.54  -4369.04      38.5
ASIANPAINT.NS  9     3  -1104.20  2016.35  -3120.55      33.3
  AXISBANK.NS  9     6  11494.90  2022.32   9472.58      66.7
BAJAJ-AUTO.NS 12     4  -5157.50  2675.63  -7833.13      33.3
BAJAJFINSV.NS 10     5   5303.79  2245.15   3058.64      50.0
BAJFINANCE.NS  6     4  12389.72  1350.44  11039.28      66.7
       BEL.NS  6     1  -6557.56  1345.34  -7902.89      16.7
BHARTIARTL.NS 15     5   3400.80  3364.56     36.25      33.3
     CIPLA.NS 11     3  -2518.59  2467.04  -4985.63      27.3
 COALINDIA.NS 12     3 -11026.77  2691.48 -13718.25      25.0
   DRREDDY.NS 13 